In [1]:
include("load_s1slc_ann.jl");


In [2]:
f = open("ann_path.txt")
path = readlines(f)

1-element Array{String,1}:
 "/Users/simon/Data/Sentinel/S1B_IW_SLC__1SDV_20170315T053950_20170315T054018_004715_0083B0_7EE9.SAFE/annotation/s1b-iw3-slc-vv-20170315t053950-20170315t054018-004715-0083b0-006.xml"

In [3]:
meta = load_s1slc_ann(path[1])

Dict{String,Any} with 10 entries:
  "slant_range_time"    => 0.00602248
  "burst_count"         => 10
  "range_pixel_spacing" => 2.32956
  "t_start"             => 50.7031
  "azimuth_frequency"   => 486.486
  "t_stop"              => 78.6587
  "lines_per_burst"     => 1524
  "samples_per_burst"   => 25330
  "t_0"                 => 2017-03-15T05:39:00
  "burst_meta"          => Dict{String,Any}("first_line_mosaic"=>[1, 1343, 2684…

In [7]:
import EzXML
import XMLDict
include("convert_time.jl");

In [5]:


function load_s1_ann(path)
    ## open xml files
    doc = EzXML.readxml(path)
    meta_dict = XMLDict.xml_dict(doc)
    
    # Create empty dict for results
    s1_meta  = Dict{String,Any}()
    burst_meta  = Dict{String,Any}()
    
    # get start and stop time
    t_start = meta_dict["product"]["adsHeader"]["startTime"]
    t_stop = meta_dict["product"]["adsHeader"]["stopTime" ]
    s1_meta["t_0"] = str2date(t_start)
    s1_meta["t_start"] = str_date2float(t_start,s1_meta["t_0"])
    s1_meta["t_stop"] = str_date2float(t_stop,s1_meta["t_0"]);
    
    # get infor for "imageInformation"
    img_info = meta_dict["product"]["imageAnnotation"]["imageInformation"];
    s1_meta["range_pixel_spacing"] = parse(Float64,img_info["rangePixelSpacing"])
    s1_meta["azimuth_frequency"] = parse(Float64,img_info["azimuthFrequency"])
    
    # Get infor about burst
    swath_timing = meta_dict["product"]["swathTiming"]
    s1_meta["lines_per_burst"] =  parse(Int,swath_timing["linesPerBurst"])
    s1_meta["samples_per_burst"] =  parse(Int,swath_timing["samplesPerBurst"])
    s1_meta["burst_count"] = parse(Int,swath_timing["burstList"][:count])
    
    # Get info for each burst
    burst = swath_timing["burstList"]["burst"]
    burst_meta["burst_times"] = [str_date2float(elem["azimuthTime"],s1_meta["t_0"]) for elem in burst]
    burst_meta["fist_valid_pixel"] = [parse.(Int,split(elem["firstValidSample"][""])) for elem in burst]
    burst_meta["last_valid_pixel"] = [parse.(Int,split(elem["lastValidSample"][""])) for elem in burst];
    
    # create a array with info about what line the first line in each burst corrosponds to in a mosaic
    first_line_mosaic = 1 .+(burst_meta["burst_times"] .- s1_meta["t_start"]) .*s1_meta["azimuth_frequency"]
    burst_meta["first_line_mosaic"] = round.(Int,first_line_mosaic)
    
    # collect info about bursts
    s1_meta["burst_meta"] = burst_meta
    
    return s1_meta
end
    

load_s1_ann (generic function with 1 method)

In [6]:
f = open("ann_path.txt")
path = readlines(f)

1-element Array{String,1}:
 "/Users/simon/Data/Sentinel/S1B_IW_SLC__1SDV_20170315T053950_20170315T054018_004715_0083B0_7EE9.SAFE/annotation/s1b-iw3-slc-vv-20170315t053950-20170315t054018-004715-0083b0-006.xml"

In [7]:
meta = load_s1_ann(path[1]) 

Dict{String,Any} with 9 entries:
  "burst_count"         => 10
  "range_pixel_spacing" => 2.32956
  "t_start"             => 50.7031
  "azimuth_frequency"   => 486.486
  "t_stop"              => 78.6587
  "lines_per_burst"     => 1524
  "samples_per_burst"   => 25330
  "t_0"                 => 2017-03-15T05:39:00
  "burst_meta"          => Dict{String,Any}("first_line_mosaic"=>[1, 1343, 2684…

## Open dict and play around

In [1]:
f = open("ann_path.txt")
path = readlines(f)
path[1]

"/Users/simon/Data/Sentinel/S1B_IW_SLC__1SDV_20170315T053950_20170315T054018_004715_0083B0_7EE9.SAFE/annotation/s1b-iw3-slc-vv-20170315t053950-20170315t054018-004715-0083b0-006.xml"

In [8]:
doc = EzXML.readxml(path[1])
meta_dict = XMLDict.xml_dict(doc)

OrderedCollections.OrderedDict{Any,Any} with 3 entries:
  :version  => "1.0"
  :encoding => "UTF-8"
  "product" => OrderedCollections.OrderedDict{Any,Any}("adsHeader"=>OrderedColl…

In [9]:
include("convert_time.jl");

In [10]:
s1_meta  = Dict{String,Any}()

Dict{String,Any} with 0 entries

In [11]:
t_start = meta_dict["product"]["adsHeader"]["startTime"]
t_stop = meta_dict["product"]["adsHeader"]["stopTime" ]
s1_meta["t_0"] = str2date(t_start)
s1_meta["t_start"] = str_date2float(t_start,s1_meta["t_0"])
s1_meta["t_stop"] = str_date2float(t_stop,s1_meta["t_0"]);

img_info = meta_dict["product"]["imageAnnotation"]["imageInformation"];
s1_meta["range_pixel_spacing"] = parse(Float64,img_info["rangePixelSpacing"])
s1_meta["azimuth_frequency"] = parse(Float64,img_info["azimuthFrequency"])
s1_meta["incidence_angle_mid"] = parse(Float64,img_info["incidenceAngleMidSwath"])

43.593281616654

In [12]:
meta_dict["product"]["generalAnnotation" ]

OrderedCollections.OrderedDict{Any,Any} with 9 entries:
  "productInformation"      => OrderedCollections.OrderedDict{Any,Any}("pass"=>…
  "downlinkInformationList" => OrderedCollections.OrderedDict{Any,Any}(:count=>…
  "orbitList"               => OrderedCollections.OrderedDict{Any,Any}(:count=>…
  "attitudeList"            => OrderedCollections.OrderedDict{Any,Any}(:count=>…
  "rawDataAnalysisList"     => OrderedCollections.OrderedDict{Any,Any}(:count=>…
  "replicaInformationList"  => OrderedCollections.OrderedDict{Any,Any}(:count=>…
  "noiseList"               => OrderedCollections.OrderedDict{Any,Any}(:count=>…
  "terrainHeightList"       => OrderedCollections.OrderedDict{Any,Any}(:count=>…
  "azimuthFmRateList"       => OrderedCollections.OrderedDict{Any,Any}(:count=>…

In [13]:
meta_dict["product"]["adsHeader"]

OrderedCollections.OrderedDict{Any,Any} with 10 entries:
  "missionId"           => "S1B"
  "productType"         => "SLC"
  "polarisation"        => "VV"
  "mode"                => "IW"
  "swath"               => "IW3"
  "startTime"           => "2017-03-15T05:39:50.703105"
  "stopTime"            => "2017-03-15T05:40:18.658671"
  "absoluteOrbitNumber" => "4715"
  "missionDataTakeId"   => "33712"
  "imageNumber"         => "006"

In [14]:
img_info 

OrderedCollections.OrderedDict{Any,Any} with 19 entries:
  "productFirstLineUtcTime" => "2017-03-15T05:39:50.703105"
  "productLastLineUtcTime"  => "2017-03-15T05:40:18.658671"
  "ascendingNodeTime"       => "2017-03-15T05:05:57.686857"
  "anchorTime"              => "2017-03-15T05:34:48.489923"
  "productComposition"      => "Slice"
  "sliceNumber"             => "13"
  "sliceList"               => OrderedCollections.OrderedDict{Any,Any}(:count=>…
  "slantRangeTime"          => "6.022480032448950e-03"
  "pixelValue"              => "Complex"
  "outputPixels"            => "16 bit Signed Integer"
  "rangePixelSpacing"       => "2.329562e+00"
  "azimuthPixelSpacing"     => "1.384746e+01"
  "azimuthTimeInterval"     => "2.055556299999998e-03"
  "azimuthFrequency"        => "4.864863102995529e+02"
  "numberOfSamples"         => "25330"
  "numberOfLines"           => "15240"
  "zeroDopMinusAcqTime"     => "2.995119970000000e+02"
  "incidenceAngleMidSwath"  => "4.359328161665400e+01"
  "ima

In [15]:
s1_meta

Dict{String,Any} with 6 entries:
  "incidence_angle_mid" => 43.5933
  "range_pixel_spacing" => 2.32956
  "t_start"             => 50.7031
  "azimuth_frequency"   => 486.486
  "t_stop"              => 78.6587
  "t_0"                 => 2017-03-15T05:39:00

In [16]:
swath_timing = meta_dict["product"]["swathTiming"]
s1_meta["lines_per_burst"] =  parse(Int,swath_timing["linesPerBurst"])
s1_meta["samples_per_burst"] =  parse(Int,swath_timing["samplesPerBurst"])
s1_meta["burst_count"] = parse(Int,swath_timing["burstList"][:count])


10

In [17]:
burst_meta  = Dict{String,Any}()
burst = swath_timing["burstList"]["burst"]
burst_meta["burst_times"] = [str_date2float(elem["azimuthTime"],s1_meta["t_0"]) for elem in burst]
burst_meta["fist_valid_pixel"] = [parse.(Int,split(elem["firstValidSample"][""])) for elem in burst]
burst_meta["last_valid_pixel"] = [parse.(Int,split(elem["lastValidSample"][""])) for elem in burst];
first_line_mosaic = 1 .+(burst_meta["burst_times"] .- s1_meta["t_start"]) .*s1_meta["azimuth_frequency"]
burst_meta["first_line_mosaic"] = round.(Int,first_line_mosaic)
burst_meta["last_line_mosaic"] = burst_meta["first_line_mosaic"].+s1_meta["lines_per_burst"].-1 
s1_meta["burst_meta"] = burst_meta;






In [31]:
burst_meta["burst_times"]

10-element Array{Float64,1}:
 50.703105
 53.461662
 56.218163
 58.978775
 61.735276
 64.493832
 67.254444
 70.00889 
 72.767446
 75.528058

In [34]:
t_test = 57

57

In [55]:
line =  1 .+(t_test .- s1_meta["t_start"]) .*s1_meta["azimuth_frequency"]

3064.353214893703

In [56]:
line_2 =  1 .+(t_test .- burst_meta["burst_times"][3]) .*s1_meta["azimuth_frequency"]

381.352997385673

In [57]:
line_true = line_2 + (3-1)*s1_meta["lines_per_burst"]

3429.352997385673

In [58]:
line + (3-1)*s1_meta["lines_per_burst"]-( burst_meta["burst_times"][3]-s1_meta["t_start"]) .*s1_meta["azimuth_frequency"]

3429.352997385673

364.9997824919701

In [30]:
s1_meta["burst_meta"]["last_line_mosaic"][2:end].+1 .- s1_meta["burst_meta"]["first_line_mosaic"][1:end-1]

9-element Array{Int64,1}:
 2866
 2865
 2867
 2865
 2866
 2867
 2864
 2866
 2867

In [11]:
s1_meta

Dict{String,Any} with 9 entries:
  "burst_count"         => 10
  "range_pixel_spacing" => 2.32956
  "t_start"             => 50.7031
  "azimuth_frequency"   => 486.486
  "t_stop"              => 78.6587
  "lines_per_burst"     => 1524
  "samples_per_burst"   => 25330
  "t_0"                 => 2017-03-15T05:39:00
  "burst_meta"          => Dict{String,Any}("first_line_mosaic"=>[1, 1343, 2684…

In [12]:
geolocation_list = meta_dict["product"]["geolocationGrid"]["geolocationGridPointList"]["geolocationGridPoint"];

In [13]:
geolocation_list[1]

OrderedCollections.OrderedDict{Any,Any} with 9 entries:
  "azimuthTime"    => "2017-03-15T05:39:50.703188"
  "slantRangeTime" => "6.022480032448950e-03"
  "line"           => "0"
  "pixel"          => "0"
  "latitude"       => "5.666818777336966e+01"
  "longitude"      => "8.692126035601964e+00"
  "height"         => "2.409324931888545e+01"
  "incidenceAngle" => "4.116274590658306e+01"
  "elevationAngle" => "3.634029280172153e+01"

In [14]:
geolocation_dict  = Dict{String,Any}()

Dict{String,Any} with 0 entries

In [15]:
geolocation_dict["line"] = [parse(Int,elem["line"]) for elem in geolocation_list];
geolocation_dict["sample"] = [parse(Int,elem["pixel"]) for elem in geolocation_list];
geolocation_dict["latitude"] = [parse(Float64,elem["latitude"]) for elem in geolocation_list];
geolocation_dict["longitude"] = [parse(Float64,elem["longitude"]) for elem in geolocation_list];

In [16]:
maximum(geolocation_dict["latitude"])

56.7901456330701

In [17]:
minimum(geolocation_dict["latitude"])

55.02123074846773

In [18]:
s1_meta["geolocation"] = geolocation_dict

Dict{String,Any} with 4 entries:
  "line"      => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  15240, 15240, 15240, 15240, …
  "latitude"  => [56.6682, 56.6749, 56.6816, 56.6882, 56.6947, 56.7012, 56.7075…
  "longitude" => [8.69213, 8.62024, 8.54869, 8.47746, 8.40655, 8.33594, 8.26564…
  "sample"    => [0, 1267, 2534, 3801, 5068, 6335, 7602, 8869, 10136, 11403  … …

In [38]:
meta_dict["product"]["generalAnnotation"]["productInformation"]

OrderedCollections.OrderedDict{Any,Any} with 7 entries:
  "pass"                => "Descending"
  "timelinessCategory"  => "Fast-24h"
  "platformHeading"     => "-1.636512441741978e+02"
  "projection"          => "Slant Range"
  "rangeSamplingRate"   => "6.434523812571428e+07"
  "radarFrequency"      => "5.405000454334350e+09"
  "azimuthSteeringRate" => "1.397440818000000e+00"

In [36]:
s1_meta["mission_id"] = meta_dict["product"]["adsHeader"]["missionId"]
s1_meta["product_type"] = meta_dict["product"]["adsHeader"]["productType"]
s1_meta["polarisation"] = meta_dict["product"]["adsHeader"]["polarisation"]
s1_meta["mode"] = meta_dict["product"]["adsHeader"]["mode"]
s1_meta["swath"] = parse(Int, string(meta_dict["product"]["adsHeader"]["swath"][end]))
s1_meta["image_number"] = meta_dict["product"]["adsHeader"]["imageNumber"]
s1_meta["absolute_orbit_number"] = meta_dict["product"]["adsHeader"]["absoluteOrbitNumber"]
s1_meta["mission_data_id"] = meta_dict["product"]["adsHeader"]["missionDataTakeId"]
s1_meta["pass"] = meta_dict["product"]["generalAnnotation"]["productInformation"]["pass"]


"Descending"

In [37]:
s1_meta

Dict{String,Any} with 19 entries:
  "image_number"          => "006"
  "mode"                  => "IW"
  "mission_data_id"       => "33712"
  "burst_meta"            => Dict{String,Any}("first_line_mosaic"=>[1, 1343, 26…
  "samples_per_burst"     => 25330
  "geolocation"           => Dict{String,Any}("line"=>[0, 0, 0, 0, 0, 0, 0, 0, …
  "polarisation"          => "VV"
  "absolute_orbit_number" => "4715"
  "t_start"               => 50.7031
  "azimuth_frequency"     => 486.486
  "t_stop"                => 78.6587
  "pass"                  => "Descending"
  "mission_id"            => "S1B"
  "product_type"          => "SLC"
  "burst_count"           => 10
  "range_pixel_spacing"   => 2.32956
  "lines_per_burst"       => 1524
  "t_0"                   => 2017-03-15T05:39:00
  "swath"                 => 3

In [50]:
length(s1_meta["geolocation"]["line"])

231

In [52]:
index1 = 100 .< s1_meta["geolocation"]["line"] .<5000
index2 = 5000 .< s1_meta["geolocation"]["sample"] .<10000
index = index1 .& index2
sum(index)

12

In [54]:
lat_max = maximum(s1_meta["geolocation"]["latitude"])

56.7901456330701

In [32]:
keys(s1_meta)


Base.KeySet for a Dict{String,Any} with 10 entries. Keys:
  "burst_count"
  "range_pixel_spacing"
  "t_start"
  "azimuth_frequency"
  "t_stop"
  "lines_per_burst"
  "samples_per_burst"
  "t_0"
  "burst_meta"
  "geolocation"

In [74]:
first_line_mosaic

10-element Array{Int64,1}:
     1
  1343
  2684
  4027
  5368
  6710
  8053
  9393
 10735
 12078

In [77]:
ovelap = last_line_mosaic[1:end-1]-first_line_mosaic[2:end]

9-element Array{Int64,1}:
 181
 182
 180
 182
 181
 180
 183
 181
 180

In [4]:
s1_meta  = Dict("a" => 1, "b" => 2, "c" => 3)

Dict{String,Int64} with 3 entries:
  "c" => 3
  "b" => 2
  "a" => 1

## Times

In [ ]:
doc = EzXML.readxml(path[1])
pod_dict = XMLDict.xml_dict(doc)

In [4]:
pod_dict["product"]["imageAnnotation"]["imageInformation"]

OrderedCollections.OrderedDict{Any,Any} with 19 entries:
  "productFirstLineUtcTime" => "2017-03-15T05:39:50.703105"
  "productLastLineUtcTime"  => "2017-03-15T05:40:18.658671"
  "ascendingNodeTime"       => "2017-03-15T05:05:57.686857"
  "anchorTime"              => "2017-03-15T05:34:48.489923"
  "productComposition"      => "Slice"
  "sliceNumber"             => "13"
  "sliceList"               => OrderedCollections.OrderedDict{Any,Any}(:count=>…
  "slantRangeTime"          => "6.022480032448950e-03"
  "pixelValue"              => "Complex"
  "outputPixels"            => "16 bit Signed Integer"
  "rangePixelSpacing"       => "2.329562e+00"
  "azimuthPixelSpacing"     => "1.384746e+01"
  "azimuthTimeInterval"     => "2.055556299999998e-03"
  "azimuthFrequency"        => "4.864863102995529e+02"
  "numberOfSamples"         => "25330"
  "numberOfLines"           => "15240"
  "zeroDopMinusAcqTime"     => "2.995119970000000e+02"
  "incidenceAngleMidSwath"  => "4.359328161665400e+01"
  "ima

In [5]:
slant_range_time = parse(Float64,pod_dict["product"]["imageAnnotation"]["imageInformation"]["slantRangeTime"])
range_pixel_spacing = parse(Float64,pod_dict["product"]["imageAnnotation"]["imageInformation"]["rangePixelSpacing"])
azimuth_pixel_spacing = parse(Float64,pod_dict["product"]["imageAnnotation"]["imageInformation"]["azimuthPixelSpacing"])
azimuth_time_interval = parse(Float64,pod_dict["product"]["imageAnnotation"]["imageInformation"]["azimuthTimeInterval"])
number_samples = parse(Int,pod_dict["product"]["imageAnnotation"]["imageInformation"]["numberOfSamples"])
number_lines = parse(Int,pod_dict["product"]["imageAnnotation"]["imageInformation"]["numberOfLines"])

15240

In [6]:
prf = parse(Float64,pod_dict["product"]["imageAnnotation"]["imageInformation"]["azimuthFrequency"])

486.4863102995529

In [7]:
adc_sampling_rate = parse(Float64,pod_dict["product"]["generalAnnotation"]["productInformation"]["rangeSamplingRate"])
### fs

6.434523812571428e7

In [8]:
radar_frequency = parse(Float64,pod_dict["product"]["generalAnnotation"]["productInformation"]["radarFrequency"])

5.40500045433435e9

In [9]:
6.022480032448950e-03*299792458/2

902747.0460918952

In [10]:
902747.0461 

902747.0461

In [11]:
3*10^8/2/adc_sampling_rate

2.3311748370087315

In [12]:
(number_lines-1)/prf

31.324622455699974

In [ ]:
function str2date(time)
    y = parse(Int,time[1:4])
    m = parse(Int,time[6:7])
    d = parse(Int,time[9:10])
    h = parse(Int,time[12:13])
    mi = parse(Int,time[15:16])

    return Dates.DateTime(y, m, d, h, mi)
end

In [23]:
function str_date2float(time,t_0)
    # find time diff with date time down to min
    t_i = str2date(time)
    dt_i = Dates.Second(t_i-t_0).value

    # convert seconds seperately to get float pression
    s = parse(Float64,time[18:end])

    return dt_i + s
end

str_date2float (generic function with 1 method)

In [4]:
include("convert_time.jl");

In [5]:
pod_dict["product"]["imageAnnotation"]["imageInformation"]["productFirstLineUtcTime"]

"2017-03-15T05:39:50.703105"

In [6]:
t_0 = str2date(pod_dict["product"]["imageAnnotation"]["imageInformation"]["productFirstLineUtcTime"])

2017-03-15T05:39:00

In [7]:
t_start = str_date2float(pod_dict["product"]["imageAnnotation"]["imageInformation"]["productFirstLineUtcTime"],t_0)

50.703105

In [8]:
?str_date2float

search: str_date2float



```
str_date2float(time,t_0)

# Arguments
- `time::String`: date of format of format "2017-03-15T05:39:50.703105"
- `t_0::DateTime`: Reference time

# Output
- `dt::Float64`: time relative to t_0 in seconds.
```


## Times

In [104]:
start_time = str2date(pod_dict["product"]["imageAnnotation"]["imageInformation"]["productFirstLineUtcTime"])


2017-03-15T05:39:50.703

In [105]:
pod_dict["product"]["imageAnnotation"]["imageInformation"]["productLastLineUtcTime"]

"2017-03-15T05:40:18.658671"

In [ ]:
lines_per_burst = parse(Int,pod_dict[ "product"]["swathTiming"]["linesPerBurst"])
samples_per_burst = parse(Int,pod_dict[ "product"]["swathTiming"]["samplesPerBurst"])

In [ ]:
a

In [13]:
burst = pod_dict[ "product"]["swathTiming"]["burstList"]["burst"];

In [14]:
burst[1]

OrderedCollections.OrderedDict{Any,Any} with 6 entries:
  "azimuthTime"      => "2017-03-15T05:39:50.703105"
  "azimuthAnxTime"   => "2.033016248000000e+03"
  "sensingTime"      => "2017-03-15T05:39:51.843281"
  "byteOffset"       => "122235"
  "firstValidSample" => OrderedCollections.OrderedDict{Any,Any}(:count=>"1524",…
  "lastValidSample"  => OrderedCollections.OrderedDict{Any,Any}(:count=>"1524",…